
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Dimensional Modeling and ETL
In this demo, we will explore how to implement Slowly Changing Dimensions (SCD) Type 2 using Databricks within a star schema. You will work through the process of creating dimension and fact tables, applying data transformations, and maintaining historical records using SCD Type 2. This hands-on exercise will strengthen your data modeling and ETL skills using Spark SQL and Delta Lake.

## Learning Objectives
By the end of this demo, you will be able to:
- Apply SCD Type 2 concepts to track historical changes in dimension tables using Databricks.
- Design a star schema for efficient querying and analysis in a data warehouse environment.
- Construct ETL pipelines using Spark SQL and Delta Lake to transform and load data.
- Evaluate the accuracy and completeness of data using data validation techniques.
- Develop scalable and automated data workflows using Databricks notebooks.

## Tasks to Perform

In this notebook, we will explore how to build a *dimensional model* (specifically, a star schema) from our TPC-H data, using **silver** (refined) tables and **gold** (dimension and fact) tables.

In doing so, we will walk through the following steps:

1. **Define Our Table Structures**: Understand how `GENERATED ALWAYS AS IDENTITY` works and create all necessary tables (silver and gold).  
2. **Load the Silver Layer**: Move data from `bronze` (raw) TPC-H tables to the refined tables, applying any necessary transformations (renaming columns, trimming strings, normalizing data types).  
3. **Create an SCD Type 2 Dimension**: Explore how to build a _Slowly Changing Dimension_ that preserves history for changed records.  
4. **Load the Gold Layer**: Fill the dimension and fact tables with both **initial** and **incremental** data. Demonstrate how to use a single `MERGE` statement for SCD Type 2 updates.  
5. **Validate and Explore Sample Queries**: Verify our data loads and query our new star schema for insights.  

By the end, we’ll have a **gold** star schema with an **SCD Type 2 dimension** (`DimCustomer`) that tracks historical changes.

## Prerequisites
As a reminder, you should have have already run a **setup script** in the preceding notebook that created:  
   - A user-specific catalog.  
   - Schemas: `bronze`, `silver`, and `gold`.  
   - TPC-H tables copied into `bronze`.

If, for whatever reason you have not already performed this setup, please return now to **1 - Entity Relationship Modeling and Working with Constraints** and execute the `%run` command in Cell 8 of that notebook.

## 🚨REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:
* Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**. <br>

##### **📌**If your cluster is available, select it and continue to the next cell. If the cluster is not shown:
  - In the drop-down, select **More**.
  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:
1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.
2. Find the triangle icon to the right of your compute cluster name and click it.
3. Wait a few minutes for the cluster to start.
4. Once the cluster is running, complete the steps above to select your cluster.

## Requirements

Please review the following requirements before starting the demo:

* To run this notebook, you need to use one of the following Databricks runtime(s): **15.4.x-scala2.12 15.4.x-photon-scala2.12 15.4.x-cpu-ml-scala2.12**

---

## Part 1: Table Definitions

To explore dimensional modeling, we will define:  
- **Silver** tables (sometimes called the *refined* or *integration* layer).  
- **Gold** tables that implement a **star schema** with an SCD Type 2 dimension.  

A key feature we’ll use is `GENERATED ALWAYS AS IDENTITY` in Delta, which automatically assigns incrementing numeric values for surrogate keys.

### 1.1 Set Your Catalog

In [0]:
import re

# Get the current user's email and extract the catalog name by splitting at '@' and taking the first part
user_id = spark.sql("SELECT current_user()").collect()[0][0].split("@")[0]

# Replace all special characters in the `user_id` with an underscore '_' to create the catalog name
catalog_name = re.sub(r'[^a-zA-Z0-9]', '_', user_id)

In [0]:
# Define the schema name for the silver and gold layers
silver_schema = "silver"
gold_schema = "gold"

In [0]:
# Create a widget to capture catalog name and all schema names
dbutils.widgets.text("catalog_name", catalog_name)
dbutils.widgets.text("silver_schema", silver_schema)
dbutils.widgets.text("gold_schema", gold_schema)

In [0]:
%sql
-- Set the current catalog to the extracted catalog name in DBSQL
USE CATALOG IDENTIFIER(:catalog_name);

### 1.2 Create Silver Tables

The **silver** schema is typically a place for refined data. We will define two example refined tables:  
- `refined_customer` (based on TPC-H `customer`)  
- `refined_orders` (based on TPC-H `orders`)

Each table will rename and standardize columns. We only define the schema here; we’ll load data next.

In [0]:
%sql
-- Set the current schema to the extracted silver_schema name in DBSQL
USE SCHEMA IDENTIFIER(:silver_schema);

In [0]:
%sql
-- Creating the refined_customer table if it does not already exist
CREATE TABLE IF NOT EXISTS refined_customer (
  customer_id INT,            -- Unique identifier for the customer
  name STRING,                -- Name of the customer
  address STRING,             -- Address of the customer
  nation_key INT,             -- Foreign key linking to the nation table
  phone STRING,               -- Phone number of the customer
  acct_bal DECIMAL(12, 2),    -- Account balance of the customer
  market_segment STRING,      -- Market segment of the customer
  comment STRING              -- Additional comments about the customer
);

In [0]:
%sql
-- Creating the refined_orders table if it does not already exist
CREATE TABLE IF NOT EXISTS refined_orders (
  order_id INT,                -- Unique identifier for the order
  customer_id INT,             -- Foreign key linking to the customer table
  order_status STRING,         -- Status of the order (e.g., pending, shipped)
  total_price DECIMAL(12, 2),  -- Total price of the order
  order_date DATE,             -- Date when the order was placed
  order_priority STRING,       -- Priority level of the order
  clerk STRING,                -- Clerk who handled the order
  ship_priority INT,           -- Shipping priority of the order
  comment STRING               -- Additional comments about the order
);

### 1.3 Create Gold Tables (Star Schema)

#### We define:
- **`DimCustomer`** (with SCD Type 2 attributes)  
- **`DimDate`**  
- **`FactOrders`**  

#### Key Steps
1. `GENERATED ALWAYS AS IDENTITY` for surrogate keys.  
2. Additional columns in `DimCustomer` to manage SCD Type 2 (e.g., `start_date`, `end_date`, and `is_current`).

In [0]:
%sql
-- Set the current schema to the extracted gold_schema name in DBSQL
USE SCHEMA IDENTIFIER(:gold_schema);

In [0]:
%sql
-- Create the DimCustomer table to store customer details with Slowly Changing Dimension (SCD) Type 2 attributes for historical tracking
CREATE TABLE IF NOT EXISTS DimCustomer
(
  dim_customer_key BIGINT GENERATED ALWAYS AS IDENTITY,  -- Surrogate key for the dimension table
  customer_id INT,                                       -- Unique identifier for the customer
  name STRING,                                           -- Name of the customer
  address STRING,                                        -- Address of the customer
  nation_key INT,                                        -- Foreign key linking to the nation table
  phone STRING,                                          -- Phone number of the customer
  acct_bal DECIMAL(12,2),                                -- Account balance of the customer
  market_segment STRING,                                 -- Market segment of the customer
  comment STRING,                                        -- Additional comments about the customer
  start_date DATE,                                       -- SCD2 start date indicating the beginning of the record's validity
  end_date DATE,                                         -- SCD2 end date indicating the end of the record's validity
  is_current BOOLEAN,                                    -- Flag to indicate if the record is the current version
  CONSTRAINT pk_dim_customer PRIMARY KEY (dim_customer_key)  -- Primary key constraint on the surrogate key
);

In [0]:
%sql
-- Simple DimDate table to store date-related information
CREATE TABLE IF NOT EXISTS DimDate (
  dim_date_key BIGINT GENERATED ALWAYS AS IDENTITY,  -- Surrogate key for the DimDate table
  full_date DATE,                                    -- Full date value
  day INT,                                           -- Day of the month
  month INT,                                         -- Month of the year
  year INT,                                          -- Year value
  CONSTRAINT pk_dim_date PRIMARY KEY (dim_date_key) RELY  -- Primary key constraint on dim_date_key
);

In [0]:
%sql
-- Check Current Catalog
SELECT current_catalog();

current_catalog()
labuser9889927_1744356335


In [0]:
%sql
-- Check Current Schema
SELECT current_schema();

current_schema()
gold


**Note:** Replace `<default_catalog_name>` and `<default_schema_name>` values in both the lines in the below code with the actual default catalog and schema names from the query output in the previous cells:

**Target Code Lines:**
```dbsql
CONSTRAINT fk_customer FOREIGN KEY (dim_customer_key) REFERENCES <default_catalog_name>.<default_schema_name>.DimCustomer(dim_customer_key),  -- Foreign key constraint linking to DimCustomer

CONSTRAINT fk_date FOREIGN KEY (dim_date_key) REFERENCES <default_catalog_name>.<default_schema_name>.DimDate(dim_date_key)  -- Foreign key constraint linking to DimDate
```

In [0]:
%sql
-- FactOrders table creation referencing DimCustomer and DimDate
CREATE TABLE IF NOT EXISTS FactOrders (
  fact_orders_key BIGINT GENERATED ALWAYS AS IDENTITY,  -- Surrogate key for the FactOrders table
  order_id INT,                                        -- Unique identifier for the order
  dim_customer_key BIGINT,                             -- Foreign key linking to the DimCustomer table
  dim_date_key BIGINT,                                 -- Foreign key linking to the DimDate table
  total_price DECIMAL(12, 2),                          -- Total price of the order
  order_status STRING,                                 -- Status of the order (e.g., pending, shipped)
  order_priority STRING,                               -- Priority level of the order
  clerk STRING,                                        -- Clerk who handled the order
  ship_priority INT,                                   -- Shipping priority of the order
  comment STRING,                                      -- Additional comments about the order
  CONSTRAINT pk_fact_orders PRIMARY KEY (fact_orders_key),  -- Primary key constraint on fact_orders_key
  CONSTRAINT fk_customer FOREIGN KEY (dim_customer_key) REFERENCES labuser9889927_1744356335.gold.DimCustomer(dim_customer_key),  -- Foreign key constraint linking to DimCustomer
  CONSTRAINT fk_date FOREIGN KEY (dim_date_key) REFERENCES labuser9889927_1744356335.gold.DimDate(dim_date_key)  -- Foreign key constraint linking to DimDate
);

#### Notes on `GENERATED ALWAYS AS IDENTITY`
- Each table automatically generates unique numbers for the surrogate key column.  
- You do not insert a value for those columns; Delta handles it seamlessly.

---

## Part 2: Loading Data into Silver

We will load data from the TPC-H `bronze` tables into `refined_customer` and `refined_orders`. This step assumes your TPC-H dataset is in `bronze.customer` and `bronze.orders`.

In [0]:
%sql
-- Switch to the catalog using the extracted catalog name in DBSQL
USE CATALOG IDENTIFIER(:catalog_name);

-- Switch to the silver schema in DBSQL
USE SCHEMA IDENTIFIER(:silver_schema);

In [0]:
%sql
-- Insert transformed data from the bronze.customer table into the refined_customer table
INSERT INTO
  refined_customer
SELECT
  c_custkey AS customer_id,                      -- Unique identifier for the customer
  TRIM(c_name) AS name,                          -- Name of the customer
  TRIM(c_address) AS address,                    -- Address of the customer
  c_nationkey AS nation_key,                     -- Foreign key linking to the nation table
  TRIM(c_phone) AS phone,                        -- Phone number of the customer
  CAST(c_acctbal AS DECIMAL(12, 2)) AS acct_bal, -- Account balance of the customer
  TRIM(c_mktsegment) AS market_segment,          -- Market segment of the customer
  TRIM(c_comment) AS comment                     -- Additional comments about the customer
FROM
  bronze.customer;                               -- Source table in the bronze layer

num_affected_rows,num_inserted_rows
750000,750000


In [0]:
%sql
-- Insert transformed data from the bronze.orders table into the refined_orders table
INSERT INTO
  refined_orders
SELECT
  o_orderkey AS order_id,                      -- Unique identifier for the order
  o_custkey AS customer_id,                    -- Foreign key linking to the customer table
  TRIM(o_orderstatus) AS order_status,         -- Status of the order (e.g., pending, shipped)
  CAST(o_totalprice AS DECIMAL(12, 2)) AS total_price,  -- Total price of the order
  o_orderdate AS order_date,                   -- Date when the order was placed
  TRIM(o_orderpriority) AS order_priority,     -- Priority level of the order
  TRIM(o_clerk) AS clerk,                      -- Clerk who handled the order
  o_shippriority AS ship_priority,             -- Shipping priority of the order
  TRIM(o_comment) AS comment                   -- Additional comments about the order
FROM
  bronze.orders;                               -- Source table in the bronze layer

num_affected_rows,num_inserted_rows
7500000,7500000


#### Validation
Check if records loaded into `refined_customer` and `refined_orders`:

In [0]:
%sql
-- Display the count of records in the refined_customer table
SELECT COUNT(*) AS refined_customer_count FROM refined_customer

refined_customer_count
750000


In [0]:
%sql
-- Display the count of records in the refined_orders table
SELECT COUNT(*) AS refined_orders_count FROM refined_orders

refined_orders_count
7500000


In [0]:
%sql
-- Display the count of records in the refined_customer table
SELECT COUNT(*) AS refined_customer_count FROM refined_customer

In [0]:
%sql
-- Display the count of records in the refined_orders table
SELECT COUNT(*) AS refined_orders_count FROM refined_orders

refined_orders_count
7500000


---

## Part 3: Initial Load into Gold (Dimensional Model)

#### Key Steps:
1. Perform an **initial load** of `DimCustomer` (all customers as *current*).  
2. Create date entries in `DimDate` from `refined_orders`. (This can be a pre loaded table for daily dates for multiple years)  
3. Populate `FactOrders`, linking each order to the correct dimension keys.

In [0]:
%sql
-- Switch to the gold schema using the USE SCHEMA SQL command
USE SCHEMA IDENTIFIER(:gold_schema);

### 3.1 DimCustomer (SCD Type 2) Initial Load
- Mark every row with `start_date = CURRENT_DATE()`, `end_date = NULL`, and `is_current = TRUE`.

In [0]:
%sql
-- Insert data into the DimCustomer dimension table
INSERT INTO DimCustomer
(
  customer_id,    -- Unique identifier for the customer
  name,           -- Name of the customer
  address,        -- Address of the customer
  nation_key,     -- Key representing the nation of the customer
  phone,          -- Phone number of the customer
  acct_bal,       -- Account balance of the customer
  market_segment, -- Market segment of the customer
  comment,        -- Additional comments about the customer
  start_date,     -- Start date of the record
  end_date,       -- End date of the record (NULL for current records)
  is_current      -- Flag indicating if the record is current
)
SELECT
  customer_id,    -- Select customer_id from the source table
  name,           -- Select name from the source table
  address,        -- Select address from the source table
  nation_key,     -- Select nation_key from the source table
  phone,          -- Select phone from the source table
  acct_bal,       -- Select acct_bal from the source table
  market_segment, -- Select market_segment from the source table
  `comment`,      -- Select comment from the source table
  CURRENT_DATE(), -- Set start_date to the current date
  NULL,           -- Set end_date to NULL for current records
  TRUE            -- Set is_current to TRUE for current records
FROM IDENTIFIER(:silver_schema || '.' || 'refined_customer')   -- Source table in the silver schema

num_affected_rows,num_inserted_rows
750000,750000


### 3.2 DimDate
- Collect unique `order_date` values from `refined_orders`.  
- Use built-in functions to split them into `day, month, year`.

In [0]:
%sql
-- Insert distinct dates into the DimDate dimension table
INSERT INTO DimDate
(
  full_date,  -- Full date value
  day,        -- Day part of the date
  month,      -- Month part of the date
  year        -- Year part of the date
)
SELECT DISTINCT
  order_date,          -- Full date value from refined_orders
  DAY(order_date),     -- Extracted day part of the date
  MONTH(order_date),   -- Extracted month part of the date
  YEAR(order_date)     -- Extracted year part of the date
FROM IDENTIFIER(:silver_schema || '.' || 'refined_orders')
WHERE order_date IS NOT NULL  -- Ensure the date is not null

num_affected_rows,num_inserted_rows
2406,2406


### 3.3 FactOrders
- Link each order to `DimCustomer` and `DimDate`.  
- We join on `(customer_id = dc.customer_id AND is_current = TRUE)` for SCD Type 2, ensuring we match only an active dimension record.

In [0]:
%sql
-- Insert data into the FactOrders table
INSERT INTO FactOrders
(
  order_id,          -- Unique identifier for the order
  dim_customer_key,  -- Foreign key referencing the customer dimension
  dim_date_key,      -- Foreign key referencing the date dimension
  total_price,       -- Total price of the order
  order_status,      -- Status of the order
  order_priority,    -- Priority of the order
  clerk,             -- Clerk handling the order
  ship_priority,     -- Shipping priority of the order
  comment            -- Additional comments about the order
)
SELECT
  ro.order_id,       -- Select order_id from refined_orders
  dc.dim_customer_key, -- Select dim_customer_key from DimCustomer
  dd.dim_date_key,   -- Select dim_date_key from DimDate
  ro.total_price,    -- Select total_price from refined_orders
  ro.order_status,   -- Select order_status from refined_orders
  ro.order_priority, -- Select order_priority from refined_orders
  ro.clerk,          -- Select clerk from refined_orders
  ro.ship_priority,  -- Select ship_priority from refined_orders
  ro.comment         -- Select comment from refined_orders
FROM IDENTIFIER(:silver_schema || '.' || 'refined_orders') ro
JOIN DimCustomer dc
  ON ro.customer_id = dc.customer_id
  AND dc.is_current = TRUE -- Join on customer_id and ensure the customer record is current
JOIN DimDate dd
  ON ro.order_date = dd.full_date -- Join on order_date and full_date

num_affected_rows,num_inserted_rows
7500000,7500000


#### Validation
Check record counts in each **gold** table:

In [0]:
%sql
-- Display the record count for the DimCustomer table
SELECT 'DimCustomer' AS table_name, COUNT(*) AS record_count FROM DimCustomer

table_name,record_count
DimCustomer,750000


In [0]:
%sql
-- Display the record count for the DimDate table
SELECT 'DimDate' AS table_name, COUNT(*) AS record_count FROM DimDate

table_name,record_count
DimDate,2406


In [0]:
%sql
-- Display the record count for the FactOrders table
SELECT 'FactOrders' AS table_name, COUNT(*) AS record_count FROM FactOrders

table_name,record_count
FactOrders,7500000


---

## Part 4: Incremental Updates (SCD Type 2 MERGE)

In real life, you would detect changes in `refined_customer` over time (e.g., a changed address or a new customer). When you see a difference:
1. **Close** the old record in DimCustomer (`end_date = <current date>`, `is_current = FALSE`).  
2. **Insert** a new record (with the updated attributes, `start_date = <current date>`, `end_date = NULL`, `is_current = TRUE`).  

Below is a single MERGE statement that can update the old record and insert the new version in one pass.

### 4.1 Example: Create Dummy Incremental Changes

This simulates:  
- An *existing* customer (ID=101) changing their address.  
- A *brand-new* customer (ID=99999).

In [0]:
%sql

CREATE OR REPLACE TEMP VIEW incremental_customer_updates AS
-- Existing customer with updated information
SELECT 101    AS customer_id,
       'CHANGED Name'   AS name,
       'Updated Address 500' AS address,
       77     AS nation_key,
       '555-NEW-8888'   AS phone,
       CAST(999.99 AS DECIMAL(12,2)) AS acct_bal,
       'NEW_SEGMENT'    AS market_segment,
       'Existing row changed' AS comment

UNION ALL

-- New customer with initial information
SELECT 99999,
       'Completely New',
       '123 New Street',
       99,
       '999-999-1234',
       CAST(500.00 AS DECIMAL(12,2)),
       'MARKET_NEW',
       'Newly added customer';

In [0]:
%sql
-- Display the contents of the temporary view to verify the data
SELECT * FROM incremental_customer_updates

### 4.2 Single MERGE for SCD Type 2
1. We produce two rows for *any changed* customer: one labeled `"OLD"` to close out the current record, and one labeled `"NEW"` to insert an updated version.  
2. For a truly new customer, we only produce a `"NEW"` row.

In [0]:
%sql
WITH staged_changes AS (
  -- "OLD" row: used to find and update the existing active dimension record
  SELECT
    i.customer_id,
    i.name,
    i.address,
    i.nation_key,
    i.phone,
    i.acct_bal,
    i.market_segment,
    i.comment,
    'OLD' AS row_type
  FROM incremental_customer_updates i

  UNION ALL

  -- "NEW" row: used to insert a brand-new dimension record
  SELECT
    i.customer_id,
    i.name,
    i.address,
    i.nation_key,
    i.phone,
    i.acct_bal,
    i.market_segment,
    i.comment,
    'NEW' AS row_type
  FROM incremental_customer_updates i
)

-- Perform the merge operation on the DimCustomer table
MERGE INTO DimCustomer t
USING staged_changes s
  ON t.customer_id = s.customer_id
     AND t.is_current = TRUE
     AND s.row_type = 'OLD'

-- When a match is found, update the existing record to close it out
WHEN MATCHED THEN
  UPDATE SET
    t.is_current = FALSE,
    t.end_date   = CURRENT_DATE()

-- When no match is found, insert the new record as the current version
WHEN NOT MATCHED THEN
  INSERT (
    customer_id,
    name,
    address,
    nation_key,
    phone,
    acct_bal,
    market_segment,
    comment,
    start_date,
    end_date,
    is_current
  )
  VALUES (
    s.customer_id,
    s.name,
    s.address,
    s.nation_key,
    s.phone,
    s.acct_bal,
    s.market_segment,
    s.comment,
    CURRENT_DATE(),
    NULL,
    TRUE
  );

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
4,2,0,2


#### Explanation:
- If a row is found matching the `"OLD"` record (meaning that customer currently is active in `DimCustomer`), we **update** that record to close it out (`is_current=FALSE` and `end_date=TODAY`).  
- Every `"NEW"` row won’t match, so it triggers the “INSERT” path. We insert a new row with `is_current=TRUE`, a fresh `start_date`, and no `end_date`.

### 4.3 Validate the Updated Rows

In [0]:
%sql
-- Query to display details of an existing customer with customer_id 101
SELECT 
  dim_customer_key,  -- Unique key for the customer in the dimension table
  customer_id,       -- Customer ID
  name,              -- Customer name
  address,           -- Customer address
  is_current,        -- Flag indicating if the record is current
  start_date,        -- Start date of the record
  end_date           -- End date of the record
FROM 
  DimCustomer        -- Dimension table containing customer data
WHERE 
  customer_id = 101  -- Filter condition to select the customer with ID 101
ORDER BY 
  dim_customer_key   -- Order the results by the unique customer key

dim_customer_key,customer_id,name,address,is_current,start_date,end_date
77275,101,Customer#000000101,sMmL2rNeHDltovSm Y,false,2025-04-11,2025-04-11
750003,101,CHANGED Name,Updated Address 500,true,2025-04-11,null


In [0]:
%sql
-- Query to display details of a new customer with customer_id 99999
SELECT 
  dim_customer_key,  -- Unique key for the customer in the dimension table
  customer_id,       -- Customer ID
  name,              -- Customer name
  address,           -- Customer address
  is_current,        -- Flag indicating if the record is current
  start_date,        -- Start date of the record
  end_date           -- End date of the record
FROM 
  DimCustomer        -- Dimension table containing customer data
WHERE 
  customer_id = 99999 -- Filter condition to select the customer with ID 99999
ORDER BY 
  dim_customer_key   -- Order the results by the unique customer key

dim_customer_key,customer_id,name,address,is_current,start_date,end_date
485897,99999,Customer#000099999,l6vVMVnRXQr,false,2025-04-11,2025-04-11
750004,99999,Completely New,123 New Street,true,2025-04-11,null


You should see that the old version of `customer_id = 101` now has `is_current=FALSE`, and a new version was inserted. A brand-new `customer_id=99999` has only one record (`is_current=TRUE`).

---

## Part 5: Sample Queries on the Star Schema

Now that we have `FactOrders` linking to `DimCustomer` and `DimDate`, let’s run a few checks to see how we can analyze the data.

### 5.1 Row Counts

In [0]:
%sql
-- Display the count of records in the DimCustomer table with the table name
SELECT 'DimCustomer' AS table_name, COUNT(*) AS record_count FROM DimCustomer

table_name,record_count
DimCustomer,750002


In [0]:
%sql
-- Display the count of records in the DimDate table with the table name
SELECT 'DimDate' AS table_name, COUNT(*) AS record_count FROM DimDate

table_name,record_count
DimDate,2406


In [0]:
%sql
-- Display the count of records in the FactOrders table with the table name
SELECT 'FactOrders' AS table_name, COUNT(*) AS record_count FROM FactOrders

table_name,record_count
FactOrders,7500000


### 5.2 Example Query: Top Market Segments

In [0]:
%sql
-- Display the total amount spent by customers in each market segment, limited to the top 10 segments
SELECT 
  dc.market_segment,                -- Select the market segment from the DimCustomer dimension table
  SUM(f.total_price) AS total_spent -- Calculate the total amount spent by summing the total_price from the FactOrders fact table
FROM 
  FactOrders f                      -- Fact table containing order data
JOIN 
  DimCustomer dc                    -- Dimension table containing customer data
  ON f.dim_customer_key = dc.dim_customer_key -- Join condition on customer key
GROUP BY 
  dc.market_segment                 -- Group the results by market segment
ORDER BY 
  total_spent DESC                  -- Order the results by total amount spent in descending order
LIMIT 10                            -- Limit the results to the top 10 market segments

market_segment,total_spent
FURNITURE,227766808074.12
BUILDING,227497428730.20
MACHINERY,226563021856.45
AUTOMOBILE,226311189156.13
HOUSEHOLD,225300767429.35


### 5.3 Example Query: Order Counts by Year

In [0]:
%sql
-- Count the number of orders for each year by joining the FactOrders table with the DimDate table
SELECT 
  dd.year,            -- Select the year from the DimDate dimension table
  COUNT(*) AS orders_count -- Count the number of orders for each year
FROM 
  FactOrders f        -- Fact table containing order data
JOIN 
  DimDate dd          -- Dimension table containing date data
  ON f.dim_date_key = dd.dim_date_key -- Join condition on date key
GROUP BY 
  dd.year             -- Group the results by year
ORDER BY 
  dd.year             -- Order the results by year

year,orders_count
1992,1139873
1993,1138893
1994,1137944
1995,1136308
1996,1141556
1997,1137325
1998,668101


---

## Part 6: Summary & Next Steps

1. **Defined Table Structures**:  
   - **Silver** (refined/integration) tables (`refined_customer`, `refined_orders`).  
   - **Gold** dimension/fact tables (`DimCustomer`, `DimDate`, `FactOrders`).  
   - Used `GENERATED ALWAYS AS IDENTITY` for surrogate keys.  
   - Managed SCD Type 2 in `DimCustomer` using `start_date`, `end_date`, and `is_current`.  

2. **Loaded Initial Data**:  
   - Moved from `bronze` TPC-H to **silver**.  
   - Populated **gold** dimension and fact tables.  

3. **Incremental Updates**:  
   - Demonstrated a single MERGE for SCD Type 2.  
   - Closed out old dimension records and inserted new ones simultaneously.  

4. **Validation**:  
   - Showed row counts, queries to confirm star schema functionality.  

**Where to Go Next**:  
- Incorporate automated checks for changes in `refined_customer` to keep `DimCustomer` synchronized over time.  
- Extend Type 2 logic to other dimensions, like part, supplier, or date-based attributes.  
- Create additional fact tables (e.g., FactLineItems) to make your star schema bigger.  
- Explore advanced BI or analytics queries on your star schema, taking full advantage of Databricks performance optimizations.

#### You have completed:
1. Building and structuring your **silver** tables.  
2. Designing a **gold** star schema with a Type 2 dimension.  
3. Loading your dimension and fact tables with both **initial** and **incremental** data updates.  
4. Running queries to validate and explore the star schema.

## Cleanup

Before proceeding, execute the SQL block in the cell below to clean up your work.

In [0]:
%sql
-- Drop Silver tables
DROP TABLE IF EXISTS silver.refined_customer;
DROP TABLE IF EXISTS silver.refined_orders;

-- Drop Gold tables
DROP TABLE IF EXISTS gold.DimCustomer;
DROP TABLE IF EXISTS gold.DimDate;
DROP TABLE IF EXISTS gold.FactOrders;

Remove all widgets created during the demo to clean up the notebook environment.

In [0]:
dbutils.widgets.removeAll()

## Conclusion
In this demo, we successfully implemented a star schema design for a data warehouse using Databricks. We applied the SCD Type 2 methodology to manage historical data in dimension tables while ensuring data consistency and traceability. Through step-by-step data transformation and ETL pipeline development, we gained practical experience in creating and managing data models for analytics. By the end of this demo, you will have developed proficiency in building scalable and efficient ETL pipelines, maintaining data lineage, and applying dimensional modeling concepts to real-world datasets.


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>